处理文本
====

Python 的一个强力之处就是处理文本很方便。下面就是一些例子。

字符串方法（String methods）
----

In [1]:
# 一对 三个双引号，就可以包括多行文本了
s = """
it was the best of times,
it was the worst of times,
it was the age of wisdom,
it was the age of foolishness,
it was the epoch of belief,
it was the epoch of incredulity,
it was the season of Light,
it was the season of Darkness,
it was the spring of hope,
it was the winter of despair,
"""

print s.count('of')
print s.find('wisdom')
print s.find('foolsihness')

10
72
-1


In [2]:
print s.upper()


IT WAS THE BEST OF TIMES,
IT WAS THE WORST OF TIMES,
IT WAS THE AGE OF WISDOM,
IT WAS THE AGE OF FOOLISHNESS,
IT WAS THE EPOCH OF BELIEF,
IT WAS THE EPOCH OF INCREDULITY,
IT WAS THE SEASON OF LIGHT,
IT WAS THE SEASON OF DARKNESS,
IT WAS THE SPRING OF HOPE,
IT WAS THE WINTER OF DESPAIR,



In [3]:
print s.replace('was', 'might have been')


it might have been the best of times,
it might have been the worst of times,
it might have been the age of wisdom,
it might have been the age of foolishness,
it might have been the epoch of belief,
it might have been the epoch of incredulity,
it might have been the season of Light,
it might have been the season of Darkness,
it might have been the spring of hope,
it might have been the winter of despair,



切割和拼接字符串
----

In [4]:
paths = !`echo $PATH`
print paths[0]

/bin/sh: /usr/local/bin:/Users/cliburn/git/julia/:/Developer/NVIDIA/CUDA-6.5/bin:/Users/cliburn/anaconda/bin:/usr/bin:/bin:/usr/sbin:/sbin:/usr/local/bin:/opt/X11/bin:/usr/texbin: No such file or directory


In [5]:
for path in paths[0].split(':'):
    print '=> '.join(path.strip().split('/'))

=> bin=> sh
=> usr=> local=> bin
=> Users=> cliburn=> git=> julia=> 
=> Developer=> NVIDIA=> CUDA-6.5=> bin
=> Users=> cliburn=> anaconda=> bin
=> usr=> bin
=> bin
=> usr=> sbin
=> sbin
=> usr=> local=> bin
=> opt=> X11=> bin
=> usr=> texbin
No such file or directory


# 字符串模块 string

字符串模块提供了很多 maketrans 函数（这个单词的意思没查到）。
与其解释这些函数，不如下面用一些例子来演示一下。

In [6]:
from string import maketrans

dna_to_rna = maketrans('ACTG', 'ACUG')

dna = 'gattaca'
print dna.upper().translate(dna_to_rna).lower()

gauuaca


In [7]:
# 这个 punctuation 函数的功能是去掉字符串中不想要的字符

from string import punctuation
print punctuation

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [8]:
import os

# 这里用到的是古登堡计划中的英文版爱丽丝仙境历险记（Alice in Wonderland）

if not os.path.exists('alice.txt'):
    ! wget http://www.gutenberg.org/cache/epub/11/pg11.txt -O alice.txt

In [9]:
from collections import Counter

# 删除功能 
alice = open('alice.txt').read()
words = alice.translate(None, punctuation).lower().split()
word_counts = Counter(words)
for item in word_counts.most_common(10):
    print item
print 'alice', word_counts['alice']

('the', 1804)
('and', 912)
('to', 801)
('a', 684)
('of', 625)
('it', 541)
('she', 538)
('said', 462)
('you', 429)
('in', 428)
alice 385


正则表达式（Regular expressions）
----

Regular expressions are a domain specific language for flexible text processing. It is a useful tool, but can be hard to deciper unless you use it often. Where possible, use string methods in preference to regular expressions. Sometiems, however, regular expressiosn are extreemly useful. We will illustrate its use for motif finding in DNA seqeucnes.


正则表达式是用于灵活文本处理的绝对主流工具了。这是一个很有用的工具，但是挺难掌握的，除非你经常用然后就熟练了。如果可能的话，可以更优先使用字符串方法而非正则表达式。不过有的时候，正则表达式是特别特别又用的。我们即将展示一个DNA序列中片段查找的样例。


更多内容参考 [Regular Expression HOWTO](https://docs.python.org/2/howto/regex.html) 以及 [re documnetation](https://docs.python.org/2/library/re.html)。

In [12]:
# 下面是大肠杆菌的 β-D牛乳糖酶（beta-D-galactosidase enzyme）的 DNA序列。

gene = """
>ENA|BAE76126|BAE76126.1 Escherichia coli str. K-12 substr. W3110 beta-D-galactosidase 
ATGACCATGATTACGGATTCACTGGCCGTCGTTTTACAACGTCGTGACTGGGAAAACCCT
GGCGTTACCCAACTTAATCGCCTTGCAGCACATCCCCCTTTCGCCAGCTGGCGTAATAGC
GAAGAGGCCCGCACCGATCGCCCTTCCCAACAGTTGCGCAGCCTGAATGGCGAATGGCGC
TTTGCCTGGTTTCCGGCACCAGAAGCGGTGCCGGAAAGCTGGCTGGAGTGCGATCTTCCT
GAGGCCGATACTGTCGTCGTCCCCTCAAACTGGCAGATGCACGGTTACGATGCGCCCATC
TACACCAACGTGACCTATCCCATTACGGTCAATCCGCCGTTTGTTCCCACGGAGAATCCG
ACGGGTTGTTACTCGCTCACATTTAATGTTGATGAAAGCTGGCTACAGGAAGGCCAGACG
CGAATTATTTTTGATGGCGTTAACTCGGCGTTTCATCTGTGGTGCAACGGGCGCTGGGTC
GGTTACGGCCAGGACAGTCGTTTGCCGTCTGAATTTGACCTGAGCGCATTTTTACGCGCC
GGAGAAAACCGCCTCGCGGTGATGGTGCTGCGCTGGAGTGACGGCAGTTATCTGGAAGAT
CAGGATATGTGGCGGATGAGCGGCATTTTCCGTGACGTCTCGTTGCTGCATAAACCGACT
ACACAAATCAGCGATTTCCATGTTGCCACTCGCTTTAATGATGATTTCAGCCGCGCTGTA
CTGGAGGCTGAAGTTCAGATGTGCGGCGAGTTGCGTGACTACCTACGGGTAACAGTTTCT
TTATGGCAGGGTGAAACGCAGGTCGCCAGCGGCACCGCGCCTTTCGGCGGTGAAATTATC
GATGAGCGTGGTGGTTATGCCGATCGCGTCACACTACGTCTGAACGTCGAAAACCCGAAA
CTGTGGAGCGCCGAAATCCCGAATCTCTATCGTGCGGTGGTTGAACTGCACACCGCCGAC
GGCACGCTGATTGAAGCAGAAGCCTGCGATGTCGGTTTCCGCGAGGTGCGGATTGAAAAT
GGTCTGCTGCTGCTGAACGGCAAGCCGTTGCTGATTCGAGGCGTTAACCGTCACGAGCAT
CATCCTCTGCATGGTCAGGTCATGGATGAGCAGACGATGGTGCAGGATATCCTGCTGATG
AAGCAGAACAACTTTAACGCCGTGCGCTGTTCGCATTATCCGAACCATCCGCTGTGGTAC
ACGCTGTGCGACCGCTACGGCCTGTATGTGGTGGATGAAGCCAATATTGAAACCCACGGC
ATGGTGCCAATGAATCGTCTGACCGATGATCCGCGCTGGCTACCGGCGATGAGCGAACGC
GTAACGCGAATGGTGCAGCGCGATCGTAATCACCCGAGTGTGATCATCTGGTCGCTGGGG
AATGAATCAGGCCACGGCGCTAATCACGACGCGCTGTATCGCTGGATCAAATCTGTCGAT
CCTTCCCGCCCGGTGCAGTATGAAGGCGGCGGAGCCGACACCACGGCCACCGATATTATT
TGCCCGATGTACGCGCGCGTGGATGAAGACCAGCCCTTCCCGGCTGTGCCGAAATGGTCC
ATCAAAAAATGGCTTTCGCTACCTGGAGAGACGCGCCCGCTGATCCTTTGCGAATACGCC
CACGCGATGGGTAACAGTCTTGGCGGTTTCGCTAAATACTGGCAGGCGTTTCGTCAGTAT
CCCCGTTTACAGGGCGGCTTCGTCTGGGACTGGGTGGATCAGTCGCTGATTAAATATGAT
GAAAACGGCAACCCGTGGTCGGCTTACGGCGGTGATTTTGGCGATACGCCGAACGATCGC
CAGTTCTGTATGAACGGTCTGGTCTTTGCCGACCGCACGCCGCATCCAGCGCTGACGGAA
GCAAAACACCAGCAGCAGTTTTTCCAGTTCCGTTTATCCGGGCAAACCATCGAAGTGACC
AGCGAATACCTGTTCCGTCATAGCGATAACGAGCTCCTGCACTGGATGGTGGCGCTGGAT
GGTAAGCCGCTGGCAAGCGGTGAAGTGCCTCTGGATGTCGCTCCACAAGGTAAACAGTTG
ATTGAACTGCCTGAACTACCGCAGCCGGAGAGCGCCGGGCAACTCTGGCTCACAGTACGC
GTAGTGCAACCGAACGCGACCGCATGGTCAGAAGCCGGGCACATCAGCGCCTGGCAGCAG
TGGCGTCTGGCGGAAAACCTCAGTGTGACGCTCCCCGCCGCGTCCCACGCCATCCCGCAT
CTGACCACCAGCGAAATGGATTTTTGCATCGAGCTGGGTAATAAGCGTTGGCAATTTAAC
CGCCAGTCAGGCTTTCTTTCACAGATGTGGATTGGCGATAAAAAACAACTGCTGACGCCG
CTGCGCGATCAGTTCACCCGTGCACCGCTGGATAACGACATTGGCGTAAGTGAAGCGACC
CGCATTGACCCTAACGCCTGGGTCGAACGCTGGAAGGCGGCGGGCCATTACCAGGCCGAA
GCAGCGTTGTTGCAGTGCACGGCAGATACACTTGCTGATGCGGTGCTGATTACGACCGCT
CACGCGTGGCAGCATCAGGGGAAAACCTTATTTATCAGCCGGAAAACCTACCGGATTGAT
GGTAGTGGTCAAATGGCGATTACCGTTGATGTTGAAGTGGCGAGCGATACACCGCATCCG
GCGCGGATTGGCCTGAACTGCCAGCTGGCGCAGGTAGCAGAGCGGGTAAACTGGCTCGGA
TTAGGGCCGCAAGAAAACTATCCCGACCGCCTTACTGCCGCCTGTTTTGACCGCTGGGAT
CTGCCATTGTCAGACATGTATACCCCGTACGTCTTCCCGAGCGAAAACGGTCTGCGCTGC
GGGACGCGCGAATTGAATTATGGCCCACACCAGTGGCGCGGCGACTTCCAGTTCAACATC
AGCCGCTACAGTCAACAGCAACTGATGGAAACCAGCCATCGCCATCTGCTGCACGCGGAA
GAAGGCACATGGCTGAATATCGACGGTTTCCATATGGGGATTGGTGGCGACGACTCCTGG
AGCCCGTCAGTATCGGCGGAATTCCAGCTGAGCGCCGGTCGCTACCATTACCAGTTGGTC
TGGTGTCAAAAATAA
"""

In [49]:
# 假如我们要把以 'ATA'开头的，后面跟随了1 -4个任意的核苷酸，后面跟着 'CG' 的DNA片段（motifs）, 
# 替换成同样长度的空格字符串凑齐什么的
# Suppose we want to replace motifs that start wtih 'ATA', 
# followed by between 1 and 4 of any nucleotide, followed by 'CG'
# with a blank string of the same length

from toolz import partition

def replace(match):
    return ' ' * len(match.group(0))

# convert FASTA into single DNA sequence
dna = ''.join(line for line in gene.strip().split('\n') 
              if not line.startswith('>'))
pattern = 'ATA.{1,4}CG'
modified_dna = re.sub(pattern, replace, dna)

# pretty print modified sequence
linewidth = 60
print '\n'.join([''.join(line) for line 
                in partition(linewidth, modified_dna)])

ATGACCATGATTACGGATTCACTGGCCGTCGTTTTACAACGTCGTGACTGGGAAAACCCT
GGCGTTACCCAACTTAATCGCCTTGCAGCACATCCCCCTTTCGCCAGCTGGCGTA     
 AAGAGGCCCGCACCGATCGCCCTTCCCAACAGTTGCGCAGCCTGAATGGCGAATGGCGC
TTTGCCTGGTTTCCGGCACCAGAAGCGGTGCCGGAAAGCTGGCTGGAGTGCGATCTTCCT
GAGGCCG         TCGTCCCCTCAAACTGGCAGATGCACGGTTACGATGCGCCCATC
TACACCAACGTGACCTATCCCATTACGGTCAATCCGCCGTTTGTTCCCACGGAGAATCCG
ACGGGTTGTTACTCGCTCACATTTAATGTTGATGAAAGCTGGCTACAGGAAGGCCAGACG
CGAATTATTTTTGATGGCGTTAACTCGGCGTTTCATCTGTGGTGCAACGGGCGCTGGGTC
GGTTACGGCCAGGACAGTCGTTTGCCGTCTGAATTTGACCTGAGCGCATTTTTACGCGCC
GGAGAAAACCGCCTCGCGGTGATGGTGCTGCGCTGGAGTGACGGCAGTTATCTGGAAGAT
CAGGATATGTGGCGGATGAGCGGCATTTTCCGTGACGTCTCGTTGCTGC        ACT
ACACAAATCAGCGATTTCCATGTTGCCACTCGCTTTAATGATGATTTCAGCCGCGCTGTA
CTGGAGGCTGAAGTTCAGATGTGCGGCGAGTTGCGTGACTACCTACGGGTAACAGTTTCT
TTATGGCAGGGTGAAACGCAGGTCGCCAGCGGCACCGCGCCTTTCGGCGGTGAAATTATC
GATGAGCGTGGTGGTTATGCCGATCGCGTCACACTACGTCTGAACGTCGAAAACCCGAAA
CTGTGGAGCGCCGAAATCCCGAATCTCTATCGTGCGGTGGTTGAACTGCACACCGCCGAC
GGCACGCTGATTGAAGCAGAAGCC

自然语言工具模块 NLTK toolkit
----

如果你要做自然语言的统计分析，或者对自然语言文本进行大规模的机器学习，可以参考一下 [Natural Language Toolkit](http://www.nltk.org/)。

<font color=red>练习</font>
----

**1**. 写一个函数，用于找到一个DNA序列的互补链（译者注；就是碱基互相对应的，A和T对应，C和G对应。但实际上现实中碱基远远比这个复杂，这个只是高中生物难度的。）
例如：

给出的DNA序列    ATCGTTA
函数生成的DNA序列 TAGCAAT

In [ ]:
# YOUR CODE HERE




**2**. 写一个正则表达式，匹配下面的几种文本组合样式：

- 以下格式的电话号码: (919)-1234567 (i.e. (123)-9876543 这种可以，但 234-1234567 或者 (123)-666666) 则不行
- 邮件地址 john.doe@duke.edu (i.e. steve@gmail.com 可以，但不能使不完整的 steve@gmail)
- DNA 序列，其重要含有 motif A-C-T-G 其中的杠 - 表示 0 或者 1 个其他碱基（nucleotide，可以是 A,C,T,G 中的任意一个)

In [ ]:
# YOUR CODE HERE




**3**. 去古登堡计划网站（Project Gutenbrrg）下载简奥斯汀Jane Austem的'傲慢与偏见（Pride and Prejudice）' 

- 移除所有标点符号，然后全文转换成消协
- 查询计算其中单词'married'出现的次数
- 查询计算在同一个十个单词的语段中，'daughter'和'married'这两个单词同时出现的次数

In [ ]:
# YOUR CODE HERE




**4**.去古登堡计划网站（Project Gutenbrrg）下载"The Gutenberg Webster's Unabridged Dictionary"

- 首先导出次店内的全部词汇(大概是 109561 个词)
- 从其中定义的英文单词（ *defined* English words）查找有三个以上元音的(aeiou)
- 找出所有最长的回文词 (回文词palindrome就是正向和逆向拼写都一样的单词，比如'deified')

In [ ]:
# YOUR CODE HERE


